# prepare data from wikipedia

In [1]:
#!pip install lxml
#pip install lxml
!conda install -c conda-forge lxml --yes
import lxml.html as lh

import numpy as np # library to handle data in a vectorized manner
import pandas as pd
import requests


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxml2-2.9.10             |       hb55368b_3         1.2 MB
    libxslt-1.1.34             |       hc22bd24_0         432 KB
    lxml-4.6.2                 |   py36h04a5ba7_0         1.5 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.34-hc22bd24_0
  lxml               conda-forge/linux-64::lxml-4.6.2-py36h04a5ba7_0

The following packages will be UPDATED:

  libxml2             conda-forge::libxml2-2.9.9-h13577e0_2 --> pkgs/main::libxml2-2.9.10-hb55368b_3



libxslt-1.1.34       | 432 KB    | ########

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#wiki=requests.get(url)
tables = pd.read_html(url)[0]

# examine the data to see if any neighbourhoods are 'not assigned' but Boroughs are with values.
tables1 = tables[tables.Neighbourhood=='Not assigned']
tables1


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned
...,...,...,...
174,M4Z,Not assigned,Not assigned
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned


we can see that the Neighbourhood and Borough are recorded as "not assighed" at the same time, so we don't need to do further modification but just ignore those 'not assigned ' cells.

In [3]:
tables = tables[tables.Borough!='Not assigned']
tables.reset_index(inplace=True,drop=True)
tables

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
tables.shape

(103, 3)

# Get the coordination

In [5]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder



Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2



ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [6]:
# initialize your variable to None
#lat_lng_coords = None

#for postal_code in tables['Postal Code']:
# loop until you get the coordinates
    #while(lat_lng_coords is None):
        #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        #lat_lng_coords = g.latlng
        
   # latitude = lat_lng_coords[0]
   # longitude = lat_lng_coords[1]

Geocoder took too long to response, let's just use the csv file:

In [7]:
coords=pd.read_csv('Geospatial_Coordinates.csv')
coords

tables_coord=tables.set_index('Postal Code').join(coords.set_index('Postal Code'))
#tables=tables.join(coords, on='Postal Code')
tables_coord.reset_index(inplace=True,drop=True)
tables_coord

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...
98,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Part 3 Analyze the clustering of restaurants

In [8]:
# extract those neighbourhoods with 'toronto':
toronto_data = tables_coord[tables_coord['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031
5,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,Downtown Toronto,Christie,43.669542,-79.422564
8,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [9]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0



geopy-2.1.0          | 64 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

In [10]:
#get Toronto's latitude and longtitude
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="cn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
#Log into Foursquare developer:
CLIENT_ID = 'OYSU3FBDP3Q1VDQRUZX0ZP4XCJDCODF2FE0ACCAEQBTLKZVV' # your Foursquare ID
CLIENT_SECRET = 'DFEQEBD2XUWQ0FZN2TFKCHQHUP01LTMIZMYWOLMJTM32KOI0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OYSU3FBDP3Q1VDQRUZX0ZP4XCJDCODF2FE0ACCAEQBTLKZVV
CLIENT_SECRET:DFEQEBD2XUWQ0FZN2TFKCHQHUP01LTMIZMYWOLMJTM32KOI0


In [12]:
import json
from pandas.io.json import json_normalize

### Search nearby restaurants in one specific neighborhood using "query=restaurants", to check the structure of json file

In [13]:
lIMIT=100
radius=500
neighborhood_latitude=43.650571 
neighborhood_longitude=-79.384568

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=restaurant'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

results1 = requests.get(url).json()['response']['venues']
results1


[{'id': '4ad4c05ff964a52048f720e3',
  'name': 'Hemispheres Restaurant & Bistro',
  'location': {'address': '110 Chestnut Street',
   'lat': 43.65488413420439,
   'lng': -79.38593077371578,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65488413420439,
     'lng': -79.38593077371578}],
   'distance': 492,
   'postalCode': 'M5G 1R3',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['110 Chestnut Street',
    'Toronto ON M5G 1R3',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d14e941735',
    'name': 'American Restaurant',
    'pluralName': 'American Restaurants',
    'shortName': 'American',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1609336691',
  'hasPerk': False},
 {'id': '50ca02c0245f2d4aa8c2b313',
  'name': 'Tundra Restaurant',
  'location': {'address': '145 Richmond Street West',
   'crossStreet': 'Hilton Tor

In [14]:
#aa=json_normalize(results1) 
#aa.head()

In [15]:
# function that extracts the category of the venue,逐行
#def get_category_type(row):
    #try:
        #categories_list = row['categories']
    #except:
        #categories_list = row['venue.categories']
        
   # if len(categories_list) == 0:
      #  return None
  #  else:
        #return categories_list[0]['name']

### Get the nearby restaurants in every neighborhoods using 'query=restaurant'

In [16]:
#define a function that extracts nearby venues given the coordinates of neighbourhoods:
def getNearbyRestaurants(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query=restaurant&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["venues"]
        
        for v in results:
            
            if len(v['categories'])!=0:
            
               venues_list.append((
                                                                                                                                                     
                  name, 
                  lat, 
                  lng, 
                  v['id'],
                  v['name'], 
                  v['location']['lat'], 
                  v['location']['lng'],  
                  v['categories'][0]['name']))#双括号不可少。注意结构！！！

    
    nearby_restaurants = pd.DataFrame([item for item in venues_list])
    nearby_restaurants.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'venue_id',                    
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    #return(venues_list) 
    return(nearby_restaurants)

In [17]:
toronto_rests = getNearbyRestaurants(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [18]:
toronto_rests

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,venue_id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,5b9e897ac9a5170039679335,Ryan Restaurant,43.655724,-79.364129,Ethiopian Restaurant
1,"Regent Park, Harbourfront",43.65426,-79.360636,4f872713e4b0abaa00979185,Site Of Great Canary Restaurant,43.653323,-79.357883,Breakfast Spot
2,"Regent Park, Harbourfront",43.65426,-79.360636,4ae5b91ff964a520a6a121e3,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,5bf9f9f7135b39002cd82007,Weheliye Restaurant,43.658536,-79.365689,African Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,540bea73498e234757c8b0a7,White Elephant,43.655884,-79.363586,Thai Restaurant
...,...,...,...,...,...,...,...,...
480,Church and Wellesley,43.66586,-79.383160,4c0e3ec52466a59312b07821,Subway,43.667527,-79.388919,Sandwich Place
481,Church and Wellesley,43.66586,-79.383160,4b9ae915f964a5201ae235e3,Mullins Irish Alehouse,43.666303,-79.387936,Pub
482,Church and Wellesley,43.66586,-79.383160,4ae787b9f964a52020ac21e3,Tokyo Kitchen,43.668783,-79.385153,Japanese Restaurant
483,Church and Wellesley,43.66586,-79.383160,4ae1b52bf964a520dd8621e3,Daily Sushi,43.661385,-79.381580,Sushi Restaurant


In [19]:
print(toronto_rests.shape)


(485, 8)


In [20]:
toronto_rests.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,venue_id,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Berczy Park,12,12,12,12,12,12,12
"Brockton, Parkdale Village, Exhibition Place",4,4,4,4,4,4,4
Central Bay Street,41,41,41,41,41,41,41
Christie,3,3,3,3,3,3,3
Church and Wellesley,19,19,19,19,19,19,19
"Commerce Court, Victoria Hotel",41,41,41,41,41,41,41
Davisville,4,4,4,4,4,4,4
Davisville North,4,4,4,4,4,4,4
"Dufferin, Dovercourt Village",1,1,1,1,1,1,1


In [21]:
print('There are {} uniques categories.'.format(len(toronto_rests['Venue Category'].unique())))

There are 55 uniques categories.


In [22]:
#toronto_rests
rest_onehot = pd.get_dummies(toronto_rests[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe,add a space and it works!!!!
rest_onehot['Neighborhood'] = toronto_rests['Neighborhood'] 
rest_onehot

# move neighborhood column to the first column
fixed_columns = [rest_onehot.columns[-1]] + list(rest_onehot.columns[:-1])
rest_onehot = rest_onehot[fixed_columns]

rest_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Asian Restaurant,Bar,Beer Bar,Breakfast Spot,Brewery,Cantonese Restaurant,Caribbean Restaurant,...,Restaurant,Sandwich Place,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [23]:
rest_grouped = rest_onehot.groupby('Neighborhood').mean().reset_index()
rest_grouped

,Neighborhood,African Restaurant,American Restaurant,Asian Restaurant,Bar,Beer Bar,Breakfast Spot,Brewery,Cantonese Restaurant,Caribbean Restaurant,...,Restaurant,Sandwich Place,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.166667,0.000000,0.083333,0.083333,0.083333,0.000000,0.000000,0.000000,...,0.083333,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.250000,0.000000
2,Central Bay Street,0.000000,0.048780,0.048780,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,...,0.073171,0.073171,0.000000,0.048780,0.04878,0.024390,0.024390,0.000000,0.024390,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.333333,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.052632,0.105263,0.000000,0.157895,0.00000,0.052632,0.000000,0.000000,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.048780,0.024390,0.048780,0.024390,0.024390,0.000000,0.000000,0.024390,...,0.268293,0.024390,0.024390,0.024390,0.00000,0.000000,0.000000,0.000000,0.000000,0.024390
6,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.250000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Davisville North,0.000000,0.000000,0.250000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,...,0.250000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"First Canadian Place, Underground city",0.000000,0.026316,0.026316,0.052632,0.000000,0.026316,0.000000,0.000000,0.026316,...,0.289474,0.026316,0.000000,0.026316,0.00000,0.000000,0.000000,0.000000,0.000000,0.026316


In [24]:
#define a function to sort the restaurant types 
def return_most_common_rests(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]#return index!

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = rest_grouped['Neighborhood']

for ind in np.arange(rest_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_rests(rest_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Diner,American Restaurant,Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Pub,Japanese Restaurant,Breakfast Spot,Beer Bar,Bar
1,"Brockton, Parkdale Village, Exhibition Place",Vietnamese Restaurant,Japanese Restaurant,Bar,Breakfast Spot,Eastern European Restaurant,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment
2,Central Bay Street,Chinese Restaurant,Restaurant,Sandwich Place,Asian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Szechuan Restaurant
3,Christie,Nightclub,Restaurant,Japanese Restaurant,Wine Bar,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment,Gastropub,French Restaurant
4,Church and Wellesley,Sushi Restaurant,Fast Food Restaurant,Diner,Sandwich Place,New American Restaurant,Pub,Italian Restaurant,Japanese Restaurant,Movie Theater,Chinese Restaurant


In [26]:
from sklearn.cluster import KMeans
import random

kclusters = 4

rest_grouped_clustering = rest_grouped.drop('Neighborhood', 1)
#toronto_grouped_clustering
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(rest_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 2, 1, 1, 0], dtype=int32)

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
merged=pd.concat([toronto_merged,neighborhoods_venues_sorted],axis=1,join='inner')
#toronto_merged['Cluster Labels']=(toronto_merged['Cluster Labels']).astype(int)
merged # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Berczy Park,Diner,American Restaurant,Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Pub,Japanese Restaurant,Breakfast Spot,Beer Bar,Bar
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,"Brockton, Parkdale Village, Exhibition Place",Vietnamese Restaurant,Japanese Restaurant,Bar,Breakfast Spot,Eastern European Restaurant,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Central Bay Street,Chinese Restaurant,Restaurant,Sandwich Place,Asian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Szechuan Restaurant
3,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Christie,Nightclub,Restaurant,Japanese Restaurant,Wine Bar,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment,Gastropub,French Restaurant
4,East Toronto,The Beaches,43.676357,-79.293031,1,Church and Wellesley,Sushi Restaurant,Fast Food Restaurant,Diner,Sandwich Place,New American Restaurant,Pub,Italian Restaurant,Japanese Restaurant,Movie Theater,Chinese Restaurant
5,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,"Commerce Court, Victoria Hotel",Restaurant,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,American Restaurant,Indian Restaurant,Bar,Diner,Wine Bar,Caribbean Restaurant
6,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Davisville,General Entertainment,Sandwich Place,Chinese Restaurant,Music Venue,Wine Bar,Dumpling Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,Gastropub
7,Downtown Toronto,Christie,43.669542,-79.422564,1,Davisville North,Breakfast Spot,Asian Restaurant,Restaurant,Wine Bar,Eastern European Restaurant,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment
8,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,3,"Dufferin, Dovercourt Village",Portuguese Restaurant,Wine Bar,Dumpling Restaurant,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment,Gastropub,French Restaurant
9,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,"First Canadian Place, Underground city",Restaurant,Japanese Restaurant,Italian Restaurant,New American Restaurant,Fast Food Restaurant,Indian Restaurant,Bar,Sushi Restaurant,Greek Restaurant,American Restaurant


In [28]:
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighbourhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighbourhood,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Brockton, Parkdale Village, Exhibition Place","India Bazaar, The Beaches West",Restaurant,Diner,Indian Restaurant,Dumpling Restaurant,Italian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment,Gastropub,French Restaurant
26,Davisville,Studio District,Restaurant,Wine Bar,Dumpling Restaurant,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment,Gastropub,French Restaurant
31,"Summerhill West, Rathnelly, South Hill, Forest...","University of Toronto, Harbord",Sandwich Place,Restaurant,Wine Bar,Diner,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment,Gastropub,French Restaurant


In [39]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighbourhood,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Berczy Park,Diner,American Restaurant,Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Pub,Japanese Restaurant,Breakfast Spot,Beer Bar,Bar
1,"Queen's Park, Ontario Provincial Government","Brockton, Parkdale Village, Exhibition Place",Vietnamese Restaurant,Japanese Restaurant,Bar,Breakfast Spot,Eastern European Restaurant,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment
2,"Garden District, Ryerson",Central Bay Street,Chinese Restaurant,Restaurant,Sandwich Place,Asian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Szechuan Restaurant
3,St. James Town,Christie,Nightclub,Restaurant,Japanese Restaurant,Wine Bar,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment,Gastropub,French Restaurant
4,The Beaches,Church and Wellesley,Sushi Restaurant,Fast Food Restaurant,Diner,Sandwich Place,New American Restaurant,Pub,Italian Restaurant,Japanese Restaurant,Movie Theater,Chinese Restaurant
5,Berczy Park,"Commerce Court, Victoria Hotel",Restaurant,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,American Restaurant,Indian Restaurant,Bar,Diner,Wine Bar,Caribbean Restaurant
6,Central Bay Street,Davisville,General Entertainment,Sandwich Place,Chinese Restaurant,Music Venue,Wine Bar,Dumpling Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,Gastropub
7,Christie,Davisville North,Breakfast Spot,Asian Restaurant,Restaurant,Wine Bar,Eastern European Restaurant,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment
9,"Dufferin, Dovercourt Village","First Canadian Place, Underground city",Restaurant,Japanese Restaurant,Italian Restaurant,New American Restaurant,Fast Food Restaurant,Indian Restaurant,Bar,Sushi Restaurant,Greek Restaurant,American Restaurant
10,"Harbourfront East, Union Station, Toronto Islands","Forest Hill North & West, Forest Hill Road Park",Sushi Restaurant,Eastern European Restaurant,Wine Bar,Dumpling Restaurant,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment,Gastropub


In [40]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighbourhood,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,"Runnymede, Swansea",The Beaches,Asian Restaurant,Wine Bar,Dumpling Restaurant,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment,Gastropub,French Restaurant


In [41]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Neighbourhood,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Richmond, Adelaide, King","Dufferin, Dovercourt Village",Portuguese Restaurant,Wine Bar,Dumpling Restaurant,Italian Restaurant,Indian Restaurant,Ice Cream Shop,Greek Restaurant,General Entertainment,Gastropub,French Restaurant


url = 'https://api.foursquare.com/v2/venues/4b5c7d3cf964a520653229e3?client_id={}&client_secret={}&v={}'.format(
            
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION 
            )
            
        # make the GET request
results = requests.get(url).json()["response"]
results

#define a function that extracts nearby venues given the coordinates of neighbourhoods:
def getVenuesGrade(v_id):
    
    
    
    for venueid in v_id:
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
            venueid,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION
            )
            
        # make the GET request
        results = requests.get(url).json()#["response"]["venue"]
        
        #toronto_rests['Grade']=results['like']['count']
        
    return(results)
    #return(toronto_rests)

rest_grade=getVenuesGrade(v_id=toronto_rests['venue_id'])
rest_grade

Analyze each neighbhood:

Clustering Neighbourhoods: